In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px

import pyet
import pyeto

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

daily_data = pd.read_sql(sql='SELECT * FROM ground_data_daily', con=conn)
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
monthly_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])
geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [4]:
daily_geoinfo_data = pd.merge(daily_data, geoinfo, on='Station_ID', how='left')
monthly_geoinfo_data = pd.merge(monthly_data, geoinfo, on='Station_ID', how='left')

In [6]:
PET_Hargreaves = daily_geoinfo_data.set_index(['Date'])\
    .groupby(by="Station_ID")\
    .apply(lambda x: round(pyet.hargreaves(tmean=x['Temperature'], tmax=x['Temperature_Maximum'], tmin=x['Temperature_Minimum'], lat=pyet.utils.deg_to_rad(x['Station_Latitude'].iloc[0])), 2), include_groups=False)\
    .stack(future_stack=True).reset_index()\
    .rename(columns={0: 'PET_Hargreaves'})

daily_geoinfo_data = daily_geoinfo_data.merge(PET_Hargreaves, on=['Date', 'Station_ID'], how='left')
daily_geoinfo_data

/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)

,Station_ID,Date,Temperature_Maximum,Temperature_Minimum,Temperature,Precipitation,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,PET_Hargreaves
0,40759,2006-09-01,38.4,24.8,31.6,0.0,Sari,Mazandaran,36.536,52.998,23.0,5.90
1,40759,2006-09-02,36.6,23.0,29.8,0.0,Sari,Mazandaran,36.536,52.998,23.0,5.64
2,40759,2006-09-03,35.8,23.8,29.8,0.0,Sari,Mazandaran,36.536,52.998,23.0,5.27
3,40759,2006-09-04,34.4,22.6,28.5,0.0,Sari,Mazandaran,36.536,52.998,23.0,5.05
4,40759,2006-09-05,33.8,24.4,29.1,0.0,Sari,Mazandaran,36.536,52.998,23.0,4.54
...,...,...,...,...,...,...,...,...,...,...,...,...
156283,40719,2024-06-25,31.4,23.0,27.2,0.0,Rasht,Gilan,37.323,49.624,-8.6,5.13
156284,40719,2024-06-26,30.6,24.0,27.3,0.0,Rasht,Gilan,37.323,49.624,-8.6,4.56
156285,40719,2024-06-27,24.0,18.8,21.4,51.0,Rasht,Gilan,37.323,49.624,-8.6,3.50
156286,40719,2024-06-28,24.8,19.4,22.1,5.2,Rasht,Gilan,37.323,49.624,-8.6,3.63


In [17]:
min_count = 15

pet_monthly = (
    daily_geoinfo_data[['Station_ID', 'Date', 'PET_Hargreaves']]\
    .set_index('Date')
    .groupby(by='Station_ID')
    .resample(rule='ME')
    .agg(
        {
            'PET_Hargreaves': lambda x: x.sum(min_count=min_count),
        }
    )
).reset_index()

pet_monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Station_ID      5136 non-null   object        
 1   Date            5136 non-null   datetime64[ns]
 2   PET_Hargreaves  5134 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 120.5+ KB


In [18]:
df = pet_monthly.copy()
df['Month'] = df['Date'].dt.month
monthly_sum = df.groupby(['Station_ID', 'Month'])[['PET_Hargreaves']].sum(numeric_only=True)

def replace_with_sum(row):
    if pd.isna(row['PET_Hargreaves']):
        return monthly_sum.loc[(row['Station_ID'], row['Month']), 'PET_Hargreaves']
    else:
        return row['PET_Hargreaves']

df['PET_Hargreaves'] = df.apply(lambda row: replace_with_sum(row), axis=1)

df.drop(columns=['Month'], inplace=True)

pet_monthly = df.copy()

pet_monthly.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136 entries, 0 to 5135
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Station_ID      5136 non-null   object        
 1   Date            5136 non-null   datetime64[ns]
 2   PET_Hargreaves  5136 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 120.5+ KB


In [19]:
conn = sqlite3.connect(DATABASE_PATH)

daily_geoinfo_data[['Station_ID', 'Date', 'PET_Hargreaves']].to_sql('pet_daily', conn, if_exists='replace', index=False)
pet_monthly.to_sql('pet_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()